<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BA/blob/main/rg_rls_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://github.com/cristiandarioortegayubro/BA/blob/main/dba.png?raw=true)

![](https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png)

#**Regresión Lineal**

***La regresión lineal es un método estadístico que trata de modelar la relación entre una variable continua y una o más variables independientes mediante el ajuste de una ecuación lineal. Se llama regresión lineal simple cuando solo hay una variable independiente y regresión lineal múltiple cuando hay más de una. Dependiendo del contexto, a la variable modelada se le conoce como variable dependiente o variable respuesta, y a las variables independientes como regresores, predictores o features.***

# **Regresión Lineal Simple**

***Supóngase que un analista de recursos humanos  quiere saber si existe una relación entre el salario de los empleados y la cantidad de años de experiencia que tienen en la empresa. En caso de existir y de establecer un modelo, podría predecir el salario en virtud de los años de experiencia...***

## **Librerías**

### **Análisis de datos**

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

### **Gráficos**

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

### **Preprocesado y modelado**

#### **Con Scikit-learn**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#### **Con Statsmodels**

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

## **Conjunto de Datos**

***Para la creación del DataFrame, se arman listas con datos...***

In [ ]:
url = "https://raw.githubusercontent.com/cristiandarioortegayubro/BA/main/Datasets/salary_data.csv"

In [ ]:
datos = pd.read_csv(url, index_col=0)
datos.head()

,YearsExperience,Salary
0,1.1,39343.0
1,1.3,46205.0
2,1.5,37731.0
3,2.0,43525.0
4,2.2,39891.0


In [ ]:
datos.shape

(30, 2)

## **Representación gráfica**

In [ ]:
px.scatter(datos, 
           x=datos.YearsExperience, 
           y=datos.Salary, 
           title="Distribución de Salarios y Años de Experiencia", 
           template="gridon")

## **Correlación lineal**

In [ ]:
corr_test = pearsonr(x = datos['YearsExperience'], y =  datos['Salary'])
print("Coeficiente de correlación de Pearson: ", corr_test[0])
print("P-value: ", corr_test[1])

Coeficiente de correlación de Pearson:  0.9782416184887599
P-value:  1.1430681092271564e-20


***El test de correlación muestran una relación lineal, de intensidad mas que considerable (r = 0.978) y muy significativa (p-value = 1.143068). Tiene mucho sentido intentar generar un modelo de regresión lineal con el objetivo de predecir el salario de la persona en función de sus años de experiencia.***


In [ ]:
pearson = round(datos.corr(),4)

In [ ]:
px.imshow(pearson,
          title = "Matriz de correlacion",
          text_auto=True)

## ***Ajuste del modelo***

***Se ajusta un modelo empleando como variable respuesta Salario y como predictor Años de Experiencia. Como en todo estudio predictivo, no solo es importante ajustar el modelo, sino también cuantificar su capacidad para predecir nuevas observaciones. Para poder hacer esta evaluación, se dividen los datos en dos grupos, uno de entrenamiento y otro de test.***

### ***Scikit-learn***

#### ***División de los datos en train y test***

In [ ]:
X = datos[['YearsExperience']]
y = datos['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

In [ ]:
X.head(3)

,YearsExperience
0,1.1
1,1.3
2,1.5


In [ ]:
X.shape

(30, 1)

**En este caso X no es una serie, es un dataframe, de 30 filas por 1 columna, es decir tiene dos dimensiones**

#### ***Creación del modelo***

In [ ]:
modelo = LinearRegression()
modelo.fit(X = X_train, y = y_train)

LinearRegression()

#### ***Información del modelo***

In [ ]:
print("Coeficiente de determinación R^2:", modelo.score(X, y))

Coeficiente de determinación R^2: 0.9558054160915245


***Una vez entrenado el modelo, se evalúa la capacidad predictiva empleando el conjunto de test.***



#### **Predicción**

In [ ]:
predicciones = modelo.predict(X = X_test)
print(predicciones[0:3,])


[73017.95275371 82391.55768653 64581.70831418]


In [ ]:
X_test["Salary"] = y_test
X_test["Prediccion"] = predicciones
X_test

,YearsExperience,Salary,Prediccion
15,4.9,67938.0,73017.952754
18,5.9,81363.0,82391.557687
11,4.0,55794.0,64581.708314
4,2.2,39891.0,47709.219435
9,3.7,57189.0,61769.626834
8,3.2,64445.0,57082.824368
27,9.6,112635.0,117073.895938
3,2.0,43525.0,45834.498449


### ***Statsmodels***

***La implementación de regresión lineal de Statsmodels, es más completa que la de Scikit-learn ya que, además de ajustar el modelo, permite calcular los test estadísticos y análisis necesarios para verificar que se cumplen las condiciones sobre las que se basa este tipo de modelos.***

#### ***División de los datos en train y test***

In [ ]:
X = datos[['YearsExperience']]
y = datos['Salary']

X_train, X_test, y_train, y_test = train_test_split(X.values.reshape(-1,1),
                                                    y.values.reshape(-1,1), 
                                                    random_state=2021)

In [ ]:
X_test

array([[4.9],
       [5.9],
       [4. ],
       [2.2],
       [3.7],
       [3.2],
       [9.6],
       [2. ]])

In [ ]:
y_test

array([[ 67938.],
       [ 81363.],
       [ 55794.],
       [ 39891.],
       [ 57189.],
       [ 64445.],
       [112635.],
       [ 43525.]])

#### ***Creación del modelo utilizando matrices como en Scikit-learn***

In [ ]:
X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.OLS(endog=y_train, exog=X_train,)
modelo = modelo.fit()
print(modelo.summary());

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     460.8
Date:                Fri, 23 Sep 2022   Prob (F-statistic):           2.79e-15
Time:                        18:07:50   Log-Likelihood:                -221.21
No. Observations:                  22   AIC:                             446.4
Df Residuals:                      20   BIC:                             448.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.709e+04   2762.956      9.804      0.0

#### ***Predicción***

##### ***Predicciones con intervalo de confianza del 95%***

###### Conjunto de datos de entrenamiento

In [ ]:
predicciones = modelo.get_prediction(exog = X_train).summary_frame(alpha=0.05)
predicciones.head(4)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,116136.535445,2106.911309,111741.595466,120531.475423,103054.967315,129218.103575
1,41147.695982,2200.283763,36557.984479,45737.407485,27999.411721,54295.980243
2,63644.347821,1468.938898,60580.194974,66708.500668,50947.853817,76340.841825
3,74892.673740,1280.550631,72221.491932,77563.855549,62285.249754,87500.097727


In [ ]:
predicciones.shape

(22, 6)

###### Conjunto de datos de testeo

In [ ]:
X_test = sm.add_constant(X_test, prepend=True)

In [ ]:
predicciones_test = modelo.get_prediction(exog = X_test).summary_frame(alpha=0.05)
predicciones_test.head(10)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,73017.952754,1299.231074,70307.804225,75728.101283,60402.215265,85633.690242
1,82391.557687,1264.750007,79753.335401,85029.779972,69791.075785,94992.039588
2,64581.708314,1446.942212,61563.439749,67599.976880,51896.209804,77267.206825
3,47709.219435,1957.461314,43626.026686,51792.412185,34729.065724,60689.373146
4,61769.626834,1515.751319,58607.824987,64931.428681,49049.213388,74490.040281
5,57082.824368,1647.283667,53646.650851,60518.997885,44291.451660,69874.197076
6,117073.895938,2142.079241,112605.596940,121542.194936,103967.499665,130180.292211
7,45834.498449,2025.102865,41610.207896,50058.789002,32809.270696,58859.726201


In [ ]:
predicciones_test.shape

(8, 6)

##### ***Representación gráfica del modelo***

**Además de la línea de mínimos cuadrados, es recomendable incluir los límites superior e inferior del intervalo de confianza. Esto permite identificar la región en la que, según el modelo generado y para un determinado nivel de confianza, se encuentra el valor promedio de la variable respuesta.**



###### Conjunto de datos de entrenamiento

In [ ]:
predicciones = modelo.get_prediction(exog = X_train).summary_frame(alpha=0.05)
predicciones['x'] = X_train[:, 1]
predicciones['y'] = y_train
predicciones = predicciones.sort_values('x')

In [ ]:
predicciones.head(4)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,x,y
18,37398.254009,2345.647265,32505.319554,42291.188464,24141.077277,50655.430741,1.1,39343.0
9,39272.974996,2272.449656,34532.728078,44013.221914,26071.389175,52474.560816,1.3,46205.0
1,41147.695982,2200.283763,36557.984479,45737.407485,27999.411721,54295.980243,1.5,37731.0
8,54270.742888,1734.627593,50652.373133,57889.112643,41429.227128,67112.258648,2.9,56642.0


In [ ]:
fig = go.Figure([go.Scatter(x = predicciones.x,
                            y = predicciones.y,
                            mode = "markers",
                            showlegend = False,
                            name = "Prediccion"),
                 go.Scatter(x = predicciones.x, 
                            y = predicciones["mean"],
                            name = "OLS"),
                 go.Scatter(x = predicciones.x, 
                            y = predicciones.mean_ci_upper,
                            name = "Limite superior"),
                 go.Scatter(x = predicciones.x, 
                            y = predicciones.mean_ci_lower,
                            name = "Limite inferior"),
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Regresion lineal simple",
                  yaxis_title = "Salario",
                  xaxis_title = "Años de experiencia")

fig.show()

###### Conjunto de datos de testeo

In [ ]:
predicciones_test = modelo.get_prediction(exog = X_test).summary_frame(alpha=0.05)
predicciones_test['x'] = X_test[:, 1]
predicciones_test['y'] = y_test
predicciones_test = predicciones_test.sort_values('x')

In [ ]:
fig = go.Figure([go.Scatter(x = predicciones_test.x,
                            y = predicciones_test.y,
                            mode = "markers",
                            showlegend = False,
                            name = "Prediccion"),
                 go.Scatter(x = predicciones_test.x, 
                            y = predicciones_test["mean"],
                            mode = "lines",
                            name = "OLS"),                 
                 go.Scatter(x = predicciones_test.x, 
                            y = predicciones_test.mean_ci_upper,
                            mode = "lines",
                            name = "Limite superior"),
                 go.Scatter(x = predicciones_test.x, 
                            y = predicciones_test.mean_ci_lower,
                            mode = "lines",
                            name = "Limite inferior"),
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Regresion lineal simple",
                  yaxis_title = "Salario",
                  xaxis_title = "Años de experiencia")

fig.show()

## ***Interpretación***

***El valor de $R^2$ indica que el modelo es capaz de explicar el $95.8$% de la variabilidad observada en la variable respuesta (Salario).***

$$salario =27087 + 9373,6049 añosexperiencia $$

In [ ]:
X = datos[['YearsExperience']]
y = datos[['Salary']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

In [ ]:
px.scatter(datos, 
           x=X_train.YearsExperience, 
           y=y_train.Salary, 
           title="Distribución de Salarios y Años de Experiencia", 
           template="gridon",
           trendline="ols",
           trendline_color_override="darkorange",
           labels={"y":"Salario","x":"Años de experiencia"})

In [ ]:
px.scatter(datos, 
           x=X_test.YearsExperience, 
           y=y_test.Salary, 
           title="Distribución de Salarios y Años de Experiencia", 
           template="gridon",
           trendline="ols",
           trendline_color_override="darkorange",
           labels={"y":"Salario","x":"Años de experiencia"})